In [4]:
import torch
import numpy as np
import matplotlib.pyplot as plt
# from einops import rearrange, repeat
# import rosbags
import json
import open3d as o3d
import pandas as pd
import os
import copy
from utility import listFiles, quickVizNumpy
from scipy.spatial.transform import Rotation as R
from numpy import cos, sin, sqrt, arctan2, array
import torch.nn as nn
### fucking around
# from pretrain import loadDataFromRegisteredSlam, cart2sph, preProcess
# from train import getSamples, getUpSamples, getSpacing


2


In [4]:
name = r'building'
path = r'datasets/registered/' + name + r'.json'
data = loadDataFromRegisteredSlam(path)
training_data = preProcess(data)
np.random.shuffle(training_data)

Preparing data ... (950/971)

In [14]:
points = np.array(data['766']['point_cloud'])
quickVizNumpy(points)

In [78]:
path = r'datasets/registered/round_plant2_with_rotation.json'
# load data from path
with open(path, 'r') as file:
    data = json.load(file)
num_keys = len(data.keys())
keys = list(data.keys())

In [80]:
points = data[keys[0]]['point_cloud']
points_np = np.array(points)
mean = np.mean(points_np, axis = 0)
mean

array([11.27041196, -0.94924753, -0.19305083])

In [84]:
centre = np.array(data[keys[0]]['pose_translation'])
(mean - centre) / np.linalg.norm(mean - centre)


array([ 0.99552488, -0.09353359, -0.01347923])

In [86]:
rotation = np.array(data[keys[0]]['pose_rotation'])
rotation

array([ 0.99994928, -0.00565382, -0.0065793 ,  0.0051175 ])

In [90]:

def cart2sph(pcd_array, pose_position):
    """ Convert a n*3 point cloud, 
    with camera centre at pose_position from cartesian coordinate in global frame
    to global algined spherical coordinate at camera frame
    
    """
    pcd_local_aligned = pcd_array - pose_position
    x, y, z = pcd_local_aligned[:,0], pcd_local_aligned[:,1], pcd_local_aligned[:,2]
    XsqPlusYsq = x**2 + y**2
    r = sqrt(XsqPlusYsq + z**2)
    elev = arctan2(z, sqrt(XsqPlusYsq))
    pan = arctan2(y, x)
    output = array([r, elev, pan])
    return rearrange(output, 'a b -> b a') #take transpose


In [5]:
num_points = 10240
sample_magnitude = training_data[:num_points,0]
sample_magnitude = rearrange(sample_magnitude, 'n -> n 1')
angles = training_data[:num_points,1:3]
origins = training_data[:num_points,3:6]

In [6]:
elev = angles[:,0]
pan = angles[:,1]
### Return angles back to directions
x_tilde, y_tilde, z_tilde = cos(elev)*cos(pan), cos(elev)*sin(pan), sin(elev)      
unit_vectors = np.vstack((x_tilde, y_tilde, z_tilde))
unit_vectors = rearrange(unit_vectors, 'a b -> b a')
pos = unit_vectors*sample_magnitude + origins

In [ ]:
gt_dist_tensor = torch.tensor(ground_truth_distance)
ang_tensor = torch.tensor(angles)
origins_tensor = torch.tensor(origins)

In [ ]:
sample_pos, sample_ang, sample_org = getSamples(origins_tensor, ang_tensor, gt_dist_tensor, num_bins=100)
upsample_pos, upsample_ang, upsample_gt_distance = getUpSamples(origins_tensor, ang_tensor, gt_dist_tensor, num_rolls=0)

In [ ]:
# tile distances
gt_distance_tiled = torch.tensor(repeat(ground_truth_distance, 'b -> (b n) 1', n=100))

# stack the upsampled position to sampled positions
pos = (torch.vstack((sample_pos, upsample_pos)))
ang = (torch.vstack((sample_ang, upsample_ang)))
gt_dis = (torch.vstack((gt_distance_tiled,upsample_gt_distance)))
org = (torch.vstack((sample_org, upsample_pos)))


In [ ]:
actual_value_sigmoided = (getTargetValues(pos, gt_dis, org)).to(dtype = torch.float32)

/tmp/ipykernel_45895/3882400413.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  temp = torch.tensor((sample_positions - origins)**2)


In [ ]:
# returns pytorch tensor of sigmoid of projected SDF
def getTargetValues(sample_positions, gt_distance, origins, num_bins=100):
    # calculate distance from sample_position
    temp = torch.tensor((sample_positions - origins)**2)
    pos_distance = torch.sqrt(torch.sum(temp, dim=1, keepdim=True))
    
    # find the "projected" value
    sigmoid = nn.Sigmoid()
    values = sigmoid(-(pos_distance - gt_distance))
    return values

In [58]:
# Convert the numpy array to an Open3D PointCloud object
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(pts)

# Visualize the point cloud
o3d.visualization.draw_geometries([pcd])